# Ejercicio de repaso: BBDD Relacionales

Ejercicio sacado directamente de una entrevista de trabajo para una conocida compañía de retail.

![tablas1](../../../img/Tablas-1.png)
![tablas3](../../../img/Tablas-3.png)

In [1]:
import numpy as np
import pandas as pd
import sqlite3
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("ECI.db")
# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)



## 1. Crea un DataFrame por cada tabla

In [2]:
dict_DEPTO_CAMPANIA = {'ID_DPTO':['1','1','2', '3', '3', '3', '4', '4', '4', '4'],
                'ID_CAMPANIA':['CA1', 'CA4', 'CA2', 'CA2', 'CA5', 'CA6', 'CA1', 'CA2', 'CA3', 'CA6']}
DEPTO_CAMPANIA = pd.DataFrame(dict_DEPTO_CAMPANIA)

DATA_DEPARTAMENTOS = {
        'ID_DPTO': ['1', '2', '3', '4'],
        'DESC_DPTO': ['JOYERIA', 'BOLSOS', 'ZAPATERIA', 'BAÑO CAB.']}
DEPARTAMENTO = pd.DataFrame(DATA_DEPARTAMENTOS)

familia = [['1', '1', '2', '3', '3', '4'], 
           ['1F1', '1F2', '2F1', '3F1', '3F2', '4F1'], 
           ['PULSERAS', 'ANILLOS', 'BOLSOS DE MANO', 'NAUTICO', 'SANDALIAS', 'BERMUDAS']]

columnas = ['ID_DPTO', 'COD_FAMILIA', 'DESC_FAMILIA']
FAMILIA = pd.DataFrame(dict(zip(columnas, familia)))

dic_campanias =  { "ID_CAMPANIA" : ["CA1", "CA2", "CA3", "CA4", "CA5", "CA6"], "TIPO": ["VENTA", "VENTA", "POSVENTA", "VENTA", "POSVENTA", "POSVENTA"], "FECHA_CAMP" : ["05/02/2012", "04/02/2012", "02/10/2009", "06/05/2010", "30/11/2010", "07/02/2015"] }
CAMPANIA = pd.DataFrame(dic_campanias)

Referencias = pd.Series(['A','B','C','D','E','F','G','H'])
ID_DPTO = pd.Series(['1','2','2','2','2','3','3','4'])
COD_FAMILIA =pd.Series(['1F1','2F1','2F1','2F1','2F1','3F1','3F2','4F1'])
FECHA_ALTA = pd.Series(['21/02/2010','07/02/2014','15/05/2014','01/10/2005','31/10/2010','20/04/2001','14/02/2015','25/03/2009'])
ARTICULO = pd.DataFrame({'REFERENCIA': Referencias,'ID_DPTO': ID_DPTO,'COD_FAMILIA':COD_FAMILIA,'FECHA_ALTA':FECHA_ALTA})

talon = ['01', '02', '03', '04', '05', '06', '07', '08']
referencia = ['A', 'A', 'A', 'D', 'E', 'E', 'H', 'H']
precio = [15, 18, 14, 37.95, 125.90, 150, 22.99, 24]
fecha_venta = ['11/02/2012', '15/02/2012', '02/10/2013', '06/08/2010', '30/11/2010', '05/02/2011', '11/04/2011', '08/08/2011']
VENTA = pd.DataFrame({'TALON':talon,
                    'REFERENCIA':referencia,
                    'PRECIO':precio,
                    'FECHA_VENTA':fecha_venta})

In [3]:
DEPTO_CAMPANIA.to_sql('DEPTO_CAMPANIA', connection, if_exists='replace', index=False)
DEPARTAMENTO.to_sql('DEPARTAMENTO', connection, if_exists='replace', index=False)
FAMILIA.to_sql('FAMILIA', connection, if_exists='replace', index=False)
CAMPANIA.to_sql('CAMPANIA', connection, if_exists='replace', index=False)
ARTICULO.to_sql('ARTICULO', connection, if_exists='replace', index=False)
VENTA.to_sql('VENTA', connection, if_exists='replace', index=False)
connection.commit()

In [6]:
#Hacemos una query
sql_query("SELECT * FROM FAMILIA;")

,ID_DPTO,COD_FAMILIA,DESC_FAMILIA
0,1,1F1,PULSERAS
1,1,1F2,ANILLOS
2,2,2F1,BOLSOS DE MANO
3,3,3F1,NAUTICO
4,3,3F2,SANDALIAS
5,4,4F1,BERMUDAS


# HASTA AQUÍ

## 2. Genera una consulta que obtenga la lista ordenada de todas las referencias sin venta

In [4]:
ventas_totales=pd.merge(ARTICULO, VENTA, how="outer")
referencias_sin_venta=ventas_totales[ventas_totales["FECHA_VENTA"].isnull()]
referencias_ordenadas=referencias_sin_venta.sort_values("REFERENCIA", ascending=True).iloc[:,0:3]

referencias_ordenadas

,REFERENCIA,ID_DPTO,COD_FAMILIA
3,B,2,2F1
4,C,2,2F1
8,F,3,3F1
9,G,3,3F2


## 3. Genera una consulta que obtenga las ventas comprendidas entre 2010 y 2011 (ambos incluidos), que estén asociados a una campaña de tipo VENTA y que sean del departamento de JOYERÍA.

In [316]:
campanias_venta=pd.merge(CAMPANIA,DEPTO_CAMPANIA)
campanias_venta=campanias_venta[campanias_venta["TIPO"]=="VENTA"]
departamento_joyeria=pd.merge(campanias_venta,DEPARTAMENTO)
departamento_joyeria=departamento_joyeria[departamento_joyeria["DESC_DPTO"]=="JOYERIA"]
ventas_2010_2011=VENTA[VENTA["FECHA_VENTA"].str.contains("2010|2011")]
aritucos_2010_2011=pd.merge(ventas_2010_2011,ARTICULO, how="left")
pd.merge(aritucos_2010_2011,departamento_joyeria)

,TALON,REFERENCIA,PRECIO,FECHA_VENTA,ID_DPTO,COD_FAMILIA,FECHA_ALTA,ID_CAMPANIA,TIPO,FECHA_CAMP,DESC_DPTO


## 4. Genere una consulta que cree un campo “co_ranking” que clasifique en orden ascendente las campañas según el total de venta, siendo 1 la campaña que más ha vendido y N la que menos. (N = Total de campañas)

In [5]:
campania_departamento=pd.merge(CAMPANIA,DEPTO_CAMPANIA)[["ID_CAMPANIA","ID_DPTO"]]
campania_referencia=pd.merge(campania_departamento,ARTICULO)[["ID_CAMPANIA","REFERENCIA"]]
campania_precio=pd.merge(campania_referencia,VENTA)[["ID_CAMPANIA","REFERENCIA","PRECIO"]]
campania_total=campania_precio.groupby("ID_CAMPANIA").sum()
campania_total.sort_values("PRECIO",ascending=False)
campania_total["Co-Ranking"]=list(range(1,len(campania_total)+1))

campania_total

,PRECIO,Co-Ranking
ID_CAMPANIA,,
CA1,93.99,1
CA2,360.84,2
CA3,46.99,3
CA4,47.00,4
CA6,46.99,5


## 5. Clasifique todas las ventas en Mayor, Igual o Menor precio respecto a la media de los precios de todas las ventas realizadas.

**Resultado**:


![tablares](../../../img/Tablas-4.png)

In [318]:
lista_ranking=[]
for precio in VENTA["PRECIO"]:
    if precio > VENTA["PRECIO"].mean():
        lista_ranking.append("MAYOR")
    if precio == VENTA["PRECIO"].mean():
        lista_ranking.append("IGUAL")
    if precio < VENTA["PRECIO"].mean():
        lista_ranking.append("MENOR")

VENTA["CLASIFICACION"]=lista_ranking
VENTA.loc[:,["TALON","PRECIO","CLASIFICACION"]]

,TALON,PRECIO,CLASIFICACION
0,01,15.00,MENOR
1,02,18.00,MENOR
2,03,14.00,MENOR
3,04,37.95,MENOR
4,05,125.90,MAYOR
5,06,150.00,MAYOR
6,07,22.99,MENOR
7,08,24.00,MENOR


# 7. Genere una consulta que para cada campaña de tipo “venta” obtenga el talón medio, precio medio, nº de referencias compradas y nº de talones con venta superior al talón medio.

In [379]:
campania_venta=CAMPANIA[CAMPANIA["TIPO"]=="VENTA"]
campania_dpto=pd.merge(campania_venta,DEPTO_CAMPANIA)[["ID_CAMPANIA","ID_DPTO"]]
campania_referencia=pd.merge(campania_dpto,ARTICULO)[["ID_CAMPANIA","ID_DPTO","REFERENCIA"]]
campania_precios=pd.merge(campania_referencia,VENTA)[["ID_CAMPANIA","REFERENCIA","TALON","PRECIO"]]

precios=campania_precios.groupby("ID_CAMPANIA").mean()
n_referencia=campania_precios.groupby("ID_CAMPANIA").count()[["REFERENCIA"]]
n_talones_mayor_media=campania_precios[campania_precios["PRECIO"]>campania_precios["PRECIO"].mean()].groupby("ID_CAMPANIA").count()


In [393]:
pd_precios_refe=pd.merge(precios, n_referencia ,on="ID_CAMPANIA")
pd_final_precios=pd.merge(pd_precios_refe, n_talones_mayor_media, on="ID_CAMPANIA", how="outer").iloc[:,0:3]
pd_final_precios.


,PRECIO_x,REFERENCIA_x,REFERENCIA_y
Hola,,,
CA1,18.798000,5,NaN
CA2,72.168000,5,2.0
CA4,15.666667,3,NaN


## 8. Genere una o varias consultas que devulevan las combinaciones de familias y campañas cuya venta haya sido superior a 100€.

In [319]:
#ARTICULOS
articulo_familia=pd.merge(FAMILIA, ARTICULO)[["REFERENCIA","COD_FAMILIA","DESC_FAMILIA"]]
venta_articulo=pd.merge(articulo_familia, VENTA).iloc[:,0:6]
ventas_suma_precio=venta_articulo.groupby("COD_FAMILIA").sum()
lista_precios_float=[]
for precio in ventas_suma_precio["PRECIO"]:
    lista_precios_float.append(float(precio))
ventas_suma_precio["PRECIO_FLOAT"]=lista_precios_float
precio_formato_100=ventas_suma_precio[ventas_suma_precio["PRECIO_FLOAT"]>100]
#CAMPAÑAS:
campania_departamento=pd.merge(CAMPANIA,DEPTO_CAMPANIA)[["ID_CAMPANIA","ID_DPTO"]]
campania_referencia=pd.merge(campania_departamento,ARTICULO)[["ID_CAMPANIA","REFERENCIA"]]
campania_precio=pd.merge(campania_referencia,VENTA)[["ID_CAMPANIA","REFERENCIA","PRECIO"]]
campania_total=campania_precio.groupby("ID_CAMPANIA").sum()

In [320]:
precio_formato_100

,PRECIO,PRECIO_FLOAT
COD_FAMILIA,,
2F1,313.85,313.85


In [321]:
campania_total[campania_total["PRECIO"]>100]

,PRECIO
ID_CAMPANIA,
CA2,360.84


## 9. Genere una o varias consultas que permitan catalogar los artículos vendidos y que no son de la Campaña CA1, según si se tratan de artículos de Ropa o Accesorios. En base a los departamentos y familias. NOTA: Se valorará la elaboración de la consulta y no la veracidad del propio catálogo)

In [322]:
ls_categ_producto=[]
tabla_articulo_familia=pd.merge(ARTICULO,FAMILIA)
contador=0
for producto in tabla_articulo_familia["DESC_FAMILIA"]:
    if producto==("PULSERAS"):
        ls_categ_producto.append("Accesorios")
    if producto==("BOLSOS DE MANO"):
        ls_categ_producto.append("Accesorios")

    elif producto==("NAUTICO"):
        ls_categ_producto.append("Ropa")
    elif producto==("SANDALIAS"):
        ls_categ_producto.append("Ropa")
    elif producto==("BERMUDAS"):
        ls_categ_producto.append("Ropa")
    contador+=1
VENTA["CATEGORIA"]=ls_categ_producto
VENTA

,TALON,REFERENCIA,PRECIO,FECHA_VENTA,CLASIFICACION,CATEGORIA
0,01,A,15.00,11/02/2012,MENOR,Accesorios
1,02,A,18.00,15/02/2012,MENOR,Accesorios
2,03,A,14.00,02/10/2013,MENOR,Accesorios
3,04,D,37.95,06/08/2010,MENOR,Accesorios
4,05,E,125.90,30/11/2010,MAYOR,Accesorios
5,06,E,150.00,05/02/2011,MAYOR,Ropa
6,07,H,22.99,11/04/2011,MENOR,Ropa
7,08,H,24.00,08/08/2011,MENOR,Ropa


## 10. Genere una o varias consultas que devuelvan la variación del total de venta entre años. Tomando como variación la siguiente fórmula $$\dfrac{Year_{n} – Year_{n-1}}{Year_{n-1}}$$ (Se valorará el control de valores 0 en la división o de indeterminaciones)

![tablas5](../../../img/Tablas-5.png)

In [323]:
lista_total_anios=[]
for anio in VENTA["FECHA_VENTA"]:
    lista_total_anios.append(anio[-4:])

VENTA["ANIO_VENTA"]=lista_total_anios
ventas_por_anio=VENTA.groupby("ANIO_VENTA").sum()
ventas_por_anio

,PRECIO
ANIO_VENTA,
2010,163.85
2011,196.99
2012,33.00
2013,14.00


In [324]:
anios_unicos=set(lista_total_anios)
lista_anios_unicos=sorted(list(anios_unicos), reverse=True)
resultado_variacion={"ANIO_1":[],"ANIO_2":[],"VARIACION":[]}

lista_precios=[]
lista_variacion=[]
for precio in ventas_por_anio["PRECIO"]:
    lista_precios.append(float(precio))

index=1
while index<len(lista_precios):
    try:
        lista_variacion.append(lista_precios[index]-lista_precios[index-1])/lista_precios[index-1]
    except:
        pass
    index+=1

resultado_variacion["ANIO_1"]=lista_anios_unicos[:-1:1]
resultado_variacion["ANIO_2"]=lista_anios_unicos[1::1]
resultado_variacion["VARIACION"]=lista_variacion
pd.DataFrame(resultado_variacion)

,ANIO_1,ANIO_2,VARIACION
0,2013,2012,33.14
1,2012,2011,-163.99
2,2011,2010,-19.00
